In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import PIL
from sklearn.datasets import load_files
from glob import glob
import cv2
from google.colab import drive
import shutil
import keras
from keras.applications.vgg19 import VGG19,preprocess_input,decode_predictions
from keras.applications.vgg16 import VGG16,preprocess_input,decode_predictions
from keras.applications.xception import Xception 
from keras.applications.inception_v3 import InceptionV3 
from keras.applications.resnet50 import ResNet50 
from keras.layers import Flatten,Conv2D,BatchNormalization,Dense,Dropout
from keras.models import Model,load_model
from keras.preprocessing.image import ImageDataGenerator
import urllib
import requests
from io import BytesIO

from google.colab import drive
from google.colab import files

In [ ]:
drive.mount('/content/gdrive')

# specify the train and test image path

In [ ]:
trainpath='/content/gdrive/My Drive/Fruit classification/litchi/train'
testpath='/content/gdrive/My Drive/Fruit classification/litchi/test'
fresh=glob('/content/gdrive/My Drive/Fruit classification/litchi/test/fresh/*.jpeg')
not_fresh=glob('/content/gdrive/My Drive/Fruit classification/litchi/test/not_fresh/*.jpeg')

In [ ]:
image=cv2.imread(fresh[0])
image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
plt.imshow(image)

# VGG16 Model


In [ ]:
IMAGE_SIZE=[150,150]

In [ ]:
tl=VGG16(input_shape= IMAGE_SIZE + [3],weights='imagenet',include_top=False)

In [ ]:
for layer in tl.layers:
  layer.trainable=False

In [ ]:
x=Flatten()(tl.output)
#we can change the number of layers and nodes
x=Dense(128,activation='relu')(x)
x=Dense(32,activation='relu')(x)
prediction=Dense(2,activation='softmax')(x)
model=Model(input=tl.input,output=prediction)

In [ ]:
#op=keras.optimizers.Adam(0.0001,)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#data augmentation
train_gen=ImageDataGenerator(rescale=1.0/255,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=True,
                             vertical_flip=True,
                             rotation_range=10,
                             zoom_range=[0.8,1.2],
                             fill_mode='nearest')

test_gen=ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_set=train_gen.flow_from_directory(trainpath,
                                        target_size=(150,150),
                                        class_mode='categorical',batch_size=32)

test_set=test_gen.flow_from_directory(testpath,
                                      target_size=(150,150),
                                      class_mode='categorical',batch_size=32)

In [ ]:
r = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
)

# saving and loading model

In [ ]:
model.save('gdrive/My Drive/Fruit classification/litchi/spoilage.h5')
#model=load_model('gdrive/My Drive/Fruit classification/litchi/spoilage.h5')

In [ ]:
plt.plot(r.history['val_accuracy'],label='Val')
plt.plot(r.history['accuracy'],label='train')
plt.legend()

# Checking predictions on a random image from the test path 

In [ ]:
test=glob(testpath +'/not_fresh/*')
img=cv2.imread(np.random.choice(test))
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)

img=img/255
img=np.expand_dims(img,0)
pred=model.predict(img)
pred=np.argmax(pred)
if(pred==0):  
  print("Fresh")
else:
  print("Not-Fresh")